In [15]:
import cv2  
import numpy as np
import matplotlib.pyplot as plt
import mediapipe as mp


In [ ]:
#clase de configuracion del media pipe para detectar manos
class detectorManos():
    #define los parámetros necesarios para la detección de MediaPipe
    def __init__(self, mode=False, maxManos = 2, model_complexity=0, Confdeteccion = 0.5, Confsegui = 0.5):
        self.mode = mode
        self.maxManos = maxManos
        self.model_complexity = model_complexity
        self.Confdeteccion = Confdeteccion
        self.Confsegui = Confsegui

        self.mpmanos = mp.solutions.hands # carga modulo de mediapipe para detectar manos
        self.manos = self.mpmanos.Hands(self.mode, self.maxManos, self.model_complexity, self.Confdeteccion, self.Confsegui) 
        self.dibujo = mp.solutions.drawing_utils # herramienta para dibujar los puntos clave de la mano
        self.tip = [4, 8, 12, 16, 20] # indice de las puntas de los dedos (indice, pulgar, corazon, etc)

    #frame de video que encuentra las manos
    def encontrarManos(self, frame, dibujar = False):
        imgcolor = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        self.resultados = self.manos.process(imgcolor)

        if self.resultados.multi_hand_landmarks:
            for mano in self.resultados.multi_hand_landmarks:
                if dibujar:
                    self.dibujo.draw_landmarks(frame, mano, self.mpmanos.HAND_CONNECTIONS)
        return frame
    
    #obtiene las coordenadas de los puntos clave de la mano y devuelve lista con los detalles
    def encontrarPosicion(self, frame, ManoNum = 0, dibujar = False):
        xlista = []
        ylista = []
        bbox = []
        self.lista = []
        if self.resultados.multi_hand_landmarks:
            mi_mano = self.resultados.multi_hand_landmarks[ManoNum]
            for id, lm in enumerate(mi_mano.landmark):
                alto, ancho, c = frame.shape
                cx, cy = int(lm.x * ancho), int(lm.y * alto)
                xlista.append(cx)
                ylista.append(cy)
                self.lista.append([id, cx, cy])
                if dibujar:
                    cv2.circle(frame, (cx, cy), 5, (0, 0, 0), cv2.FILLED)
            xmin, xmax = min(xlista), max(xlista)
            ymin, ymax = min(ylista), max(ylista)
            bbox = xmin, ymin, xmax, ymax
            if dibujar:
                cv2.rectangle(frame, (xmin - 20, ymin - 20), (xmax + 20, ymax + 20), (0, 255, 0), 2)
        return self.lista, bbox
    
    #detecta los dedos arriba para cada dedo, devuelve lista con 1 si está levantado y 0 si está bajado
    def dedosArriba(self):
        dedos = []
        if self.lista[self.tip[0]][1] < self.lista[self.tip[0]-1][1]:
            dedos.append(1)
        else:
            dedos.append(0)
        
        for id in range(1,5): 
            if self.lista[self.tip[id]][2] < self.lista[self.tip[id]-2][2]:
                dedos.append(1)
            else:
                dedos.append(0)
        return dedos
    
    #Devuelve el numero total de dedos levantados
    def numeroDedos(self):
        if(len(self.lista) != 0):
            dedos = self.dedosArriba()
            return sum(dedos)

In [ ]:
import warnings
import os
import cv2
import mediapipe as mp
import numpy as np

# Ignorar advertencias de protobuf
warnings.filterwarnings("ignore", category=UserWarning, module='google.protobuf.symbol_database')

# Inicializar detector de manos y MediaPipe Face Detection
detector = detectorManos(0.75)
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.2)

# Iniciar la captura de video
vid = cv2.VideoCapture(0)

# Configuración para guardar el video de salida
output_video_path = os.path.join(os.getcwd(), 'output.mp4')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, vid.get(cv2.CAP_PROP_FPS),
                      (int(vid.get(cv2.CAP_PROP_FRAME_WIDTH)), int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))))

flag = 0

while True:
    ret, frame = vid.read()
    if not ret:
        break

    # Detectar manos en el fotograma y obtener la posición en manosInfo
    frame = detector.encontrarManos(frame)
    manosInfo, cuadro = detector.encontrarPosicion(frame, dibujar=False)

    # Detectar número de dedos
    if detector.numeroDedos() == 1:
        img = cv2.imread('caraRamon.PNG', cv2.IMREAD_UNCHANGED)  # Leer imagen con canal alfa
        flag = 1
    elif detector.numeroDedos() == 2:
        img = cv2.imread('caraNoah.PNG', cv2.IMREAD_UNCHANGED)  # Leer imagen con canal alfa
        flag = 1
    elif detector.numeroDedos() == 3:
        img = cv2.imread('caraCarlos.PNG', cv2.IMREAD_UNCHANGED)  
        flag = 1
    elif detector.numeroDedos() == 0:
        flag = 0

    if flag == 1:
        # Convertir el fotograma a formato RGB y detectar caras
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_detection.process(frame_rgb)

        if results.detections:
            for detection in results.detections:
                # Calcular la caja delimitadora de la cara recogiendo coordenadas, ancho y largo, etc
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, _ = frame.shape 
                x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)

                # Escalar la careta para ajustarla al tamaño de la cara
                escala_careta = 1.1
                overlay = cv2.resize(img, (int(w * escala_careta), int(h * escala_careta)))

                # Se calculan las coordenadas del centro de la cara y se desplaza el overlay para que coincida con el centro de la careta
                new_x = max(x + w // 2 - overlay.shape[1] // 2, 0) 
                new_y = max(y + h // 2 - overlay.shape[0] // 2, 0)

                # Compara las dimensiones del overlay con el frame para evitar que se salga del los bordes
                h_overlay, w_overlay = overlay.shape[:2]
                h_frame, w_frame = frame.shape[:2]

                # define el area del frame donde se colocará la careta (overlay)
                y1, y2 = new_y, new_y + h_overlay
                x1, x2 = new_x, new_x + w_overlay

                # Asegurarse de que las dimensiones no excedan los límites del fotograma
                y1, y2 = max(0, y1), min(h_frame, y2)
                x1, x2 = max(0, x1), min(w_frame, x2)

                #Se recorta el overlay para ajustarse la region x1, y1 a x2 y2
                overlay = overlay[0:y2 - y1, 0:x2 - x1]

                # Superponer la careta usando transparencia
                alpha_overlay = overlay[:, :, 3] / 255.0
                alpha_frame = 1.0 - alpha_overlay

                # bucle que mezcla el overlay con el frame y se suman para hacer el efecto de superposicion
                for c in range(3):  # BGR
                    frame[y1:y2, x1:x2, c] = (alpha_overlay * overlay[:, :, c] + 
                                            alpha_frame * frame[y1:y2, x1:x2, c]) 


    # Mostrar el fotograma con las caretas colocadas
    cv2.imshow('Frame', frame)

    # Guardar el fotograma en el video de salida
    out.write(frame)

    # Detener la ejecución con la tecla ESC
    if cv2.waitKey(20) == 27:
        break

# Liberar la captura de video y cerrar las ventanas
vid.release()
out.release()
cv2.destroyAllWindows()


I0000 00:00:1732140327.527124   17591 gl_context.cc:357] GL version: 2.1 (2.1 INTEL-18.7.4), renderer: Intel(R) Iris(TM) Plus Graphics 640
I0000 00:00:1732140327.539261   17591 gl_context.cc:357] GL version: 2.1 (2.1 INTEL-18.7.4), renderer: Intel(R) Iris(TM) Plus Graphics 640
W0000 00:00:1732140327.545073  280638 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1732140327.565194  280631 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1732140327.585361  280632 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [29]:
import warnings
import os


# Ignorar advertencia de protobuf
warnings.filterwarnings("ignore", category=UserWarning, module='google.protobuf.symbol_database')

# Inicializar detector de manos y MediaPipe Face Mesh
detector = detectorManos(0.75)
mp_face_mesh = mp.solutions.face_mesh
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.2)
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.2)

# Iniciar la captura de video
vid = cv2.VideoCapture(0)

# Configuración para guardar el video de salida
output_video_path = os.path.join(os.getcwd(), 'output.mp4') 
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, vid.get(cv2.CAP_PROP_FPS), 
                      (int(vid.get(cv2.CAP_PROP_FRAME_WIDTH)), int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))))

flag = 0

while True:
    ret, frame = vid.read()
    if not ret:
        break
    
    #Detectar manos en el fotograma y obtener la posicion en manosInfo
    frame = detector.encontrarManos(frame)
    manosInfo, cuadro = detector.encontrarPosicion(frame, dibujar=False)
    
    #detectar numero de dedos
    if detector.numeroDedos() == 1:
        img = cv2.imread('gafas_negras.png')
        flag = 1
    elif detector.numeroDedos() == 2:
        img = cv2.imread('gafas_verdes.png')
        flag = 1
    elif detector.numeroDedos() == 0:
        flag = 0

    if flag == 1:
        
        
        #Convierte el fotograma a formato RGB y lo procesa con los módulos de MediaPipe face_mesh y face_detection para detectar la cara 
        # y los puntos específicos como ojos
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(frame_rgb)
        results2 = face_detection.process(frame_rgb)

        if results2.detections: # Si se detecta una cara
            for detection in results2.detections:
                #Se calcula la caja delimitadora
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, _ = frame.shape
                x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)
                
        #Si se detectan puntos faciales como ojos
        if results.multi_face_landmarks:
            for landmarks in results.multi_face_landmarks:
                #Se calcula las coordenadas de los ojos y el punto medio entre ellos para colocar las gafas en el centro de la cara
                left_eye = landmarks.landmark[33] #indice 33 corresponde a ojo izq
                right_eye = landmarks.landmark[133] #indice 133 corresponde a ojo derecho
                #coordenadas de los ojos, se multiplican por el ancho y alto para obtener mejor las posiciones en pixeles de cada ojo
                left_eye_x = int(left_eye.x * iw)
                left_eye_y = int(left_eye.y * ih)
                right_eye_x = int(right_eye.x * iw)
                right_eye_y = int(right_eye.y * ih)
                #Calcula el punto medio de los ojos para usarlo como centro de colocacion de las gafas
                center_x = (left_eye_x + right_eye_x) // 2 + 55
                center_y = (left_eye_y + right_eye_y) // 2

                #Escalado de gafas y resize para tener un tamaño adecuado respecto a la cara
                escala_gafas = 1.05
                overlay = cv2.resize(img, (int(w * escala_gafas), int(h * escala_gafas)))
                new_x = max(center_x - overlay.shape[1] // 2, 0)
                new_y = max(center_y - overlay.shape[0] // 2, 0)

                #Extrae región del fotograma donde se colocaran las gafas
                n_frame = frame[new_y:new_y+overlay.shape[0], new_x:new_x+overlay.shape[1]]
                gray_overlay = cv2.cvtColor(overlay, cv2.COLOR_BGR2GRAY)
                _, mask = cv2.threshold(gray_overlay, 1, 255, cv2.THRESH_BINARY)
                mask_inv = cv2.bitwise_not(mask)

                #bg_frame es la región original del fotograma en el área de las gafas, pero haciendo un hueco con forma de gafas
                bg_frame = cv2.bitwise_and(n_frame, n_frame, mask=mask_inv)
                fg_overlay = cv2.bitwise_and(overlay, overlay, mask=mask) # contiene dsolo las gafas en si
                # Se combina el area sin gafas y con gafas
                result = cv2.add(bg_frame, fg_overlay)
                #coloca result en el frame, sobreescribiendo esa sección para que se vean las gafas en la cara detectada
                frame[new_y:new_y+overlay.shape[0], new_x:new_x+overlay.shape[1]] = result

    # Mostrar el fotograma con las gafas colocadas
    cv2.imshow('Frame', frame)
    
    # Guardar el fotograma en el video de salida
    out.write(frame)
    
    # Detener la ejecución con la tecla ESC
    if cv2.waitKey(20) == 27:
        break

# Liberar la captura de video y cerrar las ventanas
vid.release()
out.release()
cv2.destroyAllWindows()

I0000 00:00:1732134913.272179   17591 gl_context.cc:357] GL version: 2.1 (2.1 INTEL-18.7.4), renderer: Intel(R) Iris(TM) Plus Graphics 640
I0000 00:00:1732134913.291055   17591 gl_context.cc:357] GL version: 2.1 (2.1 INTEL-18.7.4), renderer: Intel(R) Iris(TM) Plus Graphics 640
I0000 00:00:1732134913.324182   17591 gl_context.cc:357] GL version: 2.1 (2.1 INTEL-18.7.4), renderer: Intel(R) Iris(TM) Plus Graphics 640
W0000 00:00:1732134913.354399  205462 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1732134913.354928  205453 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1732134913.367752  205445 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1732134913.434450  205445